In [20]:
import requests

PUBLIC_KEY = 'sdN0'
PRIVATE_KEY = 'aefec5fdsfb-4cd1-b6c3-59fa74303d41'
REGION = 'ap-southeast-1'
APP_ID = 'Wifdsawy'

In [91]:
def post_data_summary(cluster_id: str, database: str):
    url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/dataSummaries'
    auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
    headers = {'Content-Type': 'application/json'}
    payload = {
        'cluster_id': cluster_id,
        'database': database
    }
    
    response = requests.post(url, auth=auth, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text
    
def get_data_summary(cluster_id: str, database: str):
    url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/dataSummaries'
    auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
    headers = {'Content-Type': 'application/json'}
    params = {
        'cluster_id': cluster_id,
        'database': database
    }
    
    response = requests.get(url, auth=auth, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text


def get_job_status(job_id: str):
    url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/jobs/{job_id}'
    auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
    headers = {'Content-Type': 'application/json'}
    
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code == 200:
        data = response.json()
        result = data.get("result", {})

        # If 'task_tree' is present, format the response accordingly
        if "task_tree" in result:
            task_tree = result.get('task_tree', {}).get('0', {})
            task_details = {
                "Columns": task_tree.get('columns', []),
                "Rows": task_tree.get('rows', []),
                "SQL": task_tree.get('sql', 'Not available'),
                "Time Elapsed": result.get('time_elapsed', 'Not available'),
                "Status": result.get('status', 'Not available')
            }
            formatted_response = {
                "Code": data.get('code'),
                "Task Tree": task_details
            }
            return formatted_response
        else:
            # If 'task_tree' is not present, return the original 'result' part of the response
            return {"Code": data.get('code'), "Result": result}
    else:
        return {"Error": response.text, "Code": response.status_code}

# Example usage, assuming the 'cluster_id' is defined elsewhere or

    
def execute_sql(cluster_id: str, database: str, raw_question: str):
    url = f'https://{REGION}.data.tidbcloud.com/api/v1beta/app/chat2query-{APP_ID}/endpoint/v2/chat2data'
    auth = requests.auth.HTTPDigestAuth(PUBLIC_KEY, PRIVATE_KEY)
    headers = {'Content-Type': 'application/json'}
    payload = {
        'cluster_id': cluster_id,
        'database': database,
        'raw_question': raw_question
    }
    
    response = requests.post(url, auth=auth, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text


In [70]:
# Usage example
cluster_id = '10954456779882628977'
database = 'test'
result = post_data_summary(cluster_id, database)
print(result)


{'code': 200, 'msg': '', 'result': {'data_summary_id': 277707, 'job_id': 'e0255b15c03b4b6bb3c69a4773d23d6d'}}


In [92]:
# Example usage
job_id = 'e0255b15c03b4b6bb3c69a4773d23d6d'
result = get_job_status(job_id)
print(result)

{'Code': 200, 'Result': {'ended_at': 1711351186, 'job_id': 'e0255b15c03b4b6bb3c69a4773d23d6d', 'reason': '', 'result': None, 'status': 'done'}}


In [50]:
result = get_data_summary(cluster_id, database)
print(result)

{'code': 200, 'msg': '', 'result': {'data_summary_id': 277707, 'status': 'done'}}


In [81]:
result = execute_sql(cluster_id, database, "I want to see the average rating in the past 10 months")
print(result)

{'code': 200, 'msg': '', 'result': {'data_summary_id': 277707, 'job_id': 'd7ad96db70b84d2d9e9f7c649fa02c1d', 'session_context_id': 277870, 'session_id': '30db9a20663d420ab396727924007d48'}}


In [93]:
result = get_job_status('d7ad96db70b84d2d9e9f7c649fa02c1d')
print(result)

{'Code': 200, 'Result': {'ended_at': 1711351433, 'job_id': 'd7ad96db70b84d2d9e9f7c649fa02c1d', 'reason': '', 'result': {'question_id': '26db060e-9163-4f77-b6d2-493146d6f988', 'raw_question': 'I want to see the average rating in the past 10 months', 'session_context_id': 277870, 'task_tree': {'0': {'assumptions': [], 'breakdown_type': 'Resolve', 'chartOptions': {}, 'clarified_task': 'Find the average rating in the past 10 months.', 'columns': [{'col': 'average_rating'}], 'description': '', 'level': 0, 'parent_task': '', 'parent_task_id': '', 'possibleExplanations': '', 'reason': 'no proxy needed for task', 'recommendations': {}, 'rows': [['3.0676']], 'sequence_no': 0, 'sql': 'SELECT AVG(`rating`) AS `average_rating` FROM `customer_feedback` WHERE `feedback_date` >= DATE_SUB(CURDATE(), INTERVAL 10 MONTH);', 'task': 'I want to see the average rating in the past 10 months', 'task_id': '0'}}, 'time_elapsed': 7.987234354019165}, 'status': 'done'}}


Result

{'Code': 200, 'Result': {'ended_at': 1711351433, 'job_id': 'd7ad96db70b84d2d9e9f7c649fa02c1d', 'reason': '', 'result': {'question_id': '26db060e-9163-4f77-b6d2-493146d6f988', 'raw_question': 'I want to see the average rating in the past 10 months', 'session_context_id': 277870, 'task_tree': {'0': {'assumptions': [], 'breakdown_type': 'Resolve', 'chartOptions': {}, 'clarified_task': 'Find the average rating in the past 10 months.', 'columns': [{'col': 'average_rating'}], 'description': '', 'level': 0, 'parent_task': '', 'parent_task_id': '', 'possibleExplanations': '', 'reason': 'no proxy needed for task', 'recommendations': {}, 'rows': [['3.0676']], 'sequence_no': 0, 'sql': 'SELECT AVG(`rating`) AS `average_rating` FROM `customer_feedback` WHERE `feedback_date` >= DATE_SUB(CURDATE(), INTERVAL 10 MONTH);', 'task': 'I want to see the average rating in the past 10 months', 'task_id': '0'}}, 'time_elapsed': 7.987234354019165}, 'status': 'done'}}

In [ ]:
{'Code': 200, 'Result': {'ended_at': 1711351433, 'job_id': 'd7ad96db70b84d2d9e9f7c649fa02c1d', 'reason': '', 'result': {'question_id': '26db060e-9163-4f77-b6d2-493146d6f988', 'raw_question': 'I want to see the average rating in the past 10 months', 'session_context_id': 277870, 'task_tree': {'0': {'assumptions': [], 'breakdown_type': 'Resolve', 'chartOptions': {}, 'clarified_task': 'Find the average rating in the past 10 months.', 'columns': [{'col': 'average_rating'}], 'description': '', 'level': 0, 'parent_task': '', 'parent_task_id': '', 'possibleExplanations': '', 'reason': 'no proxy needed for task', 'recommendations': {}, 'rows': [['3.0676']], 'sequence_no': 0, 'sql': 'SELECT AVG(`rating`) AS `average_rating` FROM `customer_feedback` WHERE `feedback_date` >= DATE_SUB(CURDATE(), INTERVAL 10 MONTH);', 'task': 'I want to see the average rating in the past 10 months', 'task_id': '0'}}, 'time_elapsed': 7.987234354019165}, 'status': 'done'}}